# Drum Transcriber (Omnizart Edition) Colab - Ultimate Fix

Run this notebook to launch the **SOTA Omnizart-powered** Drum Transcriber Web UI.

**Fixes Included**:
- **Python 3.10**: Downgrades from Colab's default Python 3.12 to 3.10 (Compatible with libraries).
- **Robust Deps**: Installs Madmom from source, pins Numpy.
- **Nuclear Patch**: Removes all version constraints from Omnizart source.

In [ ]:
# @title 0. Install Python 3.10 (CRITICAL)
# @markdown Colab runs Python 3.12, but Omnizart/Madmom break on anything > 3.10. We MUST switch.

import sys

# 1. Install Python 3.10
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-dev -y

# 2. Set as Default
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --set python3 /usr/bin/python3.10

# 3. Install Pip for 3.10
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# 4. Check Version
!python --version

In [ ]:
# @title 1. Setup Environment (Robust Dependencies)
# @markdown Installs specific compatible versions of libraries.

import os

# 1. Install System Dependencies
!sudo apt-get install libsndfile1 ffmpeg -y

# 2. Fix Build Tools & Numpy (Crucial for Madmom)
!pip install --upgrade pip setuptools wheel
!pip install cython numpy==1.23.5

# 3. Install Madmom from Git (User request)
!pip install git+https://github.com/CPJKU/madmom.git

# 4. Install other base dependencies
!pip install pretty_midi gradio yt-dlp librosa==0.8.1 pandas plotly scikit-learn soundfile jsonschema

# 5. Clone and Patch Omnizart
if not os.path.exists("omnizart"):
    !git clone https://github.com/Music-and-Culture-Technology-Lab/omnizart.git

%cd omnizart

# --- NUCLEAR PATCHING ---
# 1. Remove pyproject.toml (It contains build constraints that block install)
if os.path.exists("pyproject.toml"):
    !rm pyproject.toml

# 2. Force delete python_requires from setup.py
!sed -i '/python_requires/d' setup.py

# 3. Relax dependencies in setup.py
!sed -i 's/PyYAML<6.0.0,>=5.3.1/PyYAML>=5.3.1/' setup.py
!sed -i 's/numba==0.48/numba>=0.48/' setup.py
!sed -i 's/librosa==0.8.0/librosa>=0.8.0/' setup.py
!sed -i 's/madmom==0.16.1/madmom>=0.16.1/' setup.py

# Install Omnizart (No dependencies, we installed them manually)
!pip install --no-deps .
%cd ..

# 6. Clone DrumTranscriber Repo
repo_url = "https://github.com/AgentHitmanFaris/DrumTranscriber.git"
repo_name = "DrumTranscriber_App"

if not os.path.exists(repo_name):
    !git clone {repo_url} {repo_name}

%cd {repo_name}

# 7. Download Omnizart checkpoints via CLI (More robust than internal API)
print("Downloading checkpoints via CLI...")
!omnizart download-checkpoints


In [ ]:
# @title 2. Run App (Omnizart Mode)
# @markdown Launches variable patching + Gradio.

launcher_code = """
import os
import gradio_app
from omnizart_wrapper import OmnizartWrapper

def load_omnizart():
    print('Loading Omnizart Wrapper...')
    return OmnizartWrapper()

gradio_app.load_model = load_omnizart
gradio_app.demo.launch(share=True)
"""

with open('run_omnizart.py', 'w') as f:
    f.write(launcher_code)

!python run_omnizart.py
